In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import EfficientNetB3

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])
  except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
    print(e)

In [3]:
# train, validation TFRecord 파일 경로
train_tfrecord_path = '../data/cat_dog/cat_dog_train.tfrecords'
valid_tfrecord_path = '../data/cat_dog/cat_dog_valid.tfrecords'

BUFFER_SIZE = 256     # 데이터 shuffle을 위한 buffer size
BATCH_SIZE = 100       # 배치 사이즈. 한번에 가져오는 이미지 데이터 개수 
NUM_CLASS = 2         # class의 개수. binary인 경우는 필요없으며 categorical인 경우 설정
IMAGE_SIZE = 150       


# TFRecord를 읽어서 데이터를 복원하기 위한 자료구조.
image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'id': tf.io.FixedLenFeature([], tf.string),
}

# 읽어들인 TFRecord를 다음의 형태(dict)로 변환하는 함수
def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, 
                                      image_feature_description)

# 위에서 얻은 ParallelMapDataset를 다음의 형태(shape)로 변환하는 함수
def map_func(target_record):
    img = target_record['image_raw']
    label = target_record['label']
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label


# 전처리(resize & augmentation) 함수
def image_resize_func(image, label):
    # result_image = image / 255
    result_image = tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))   
    return result_image, label

In [4]:
train_dataset = tf.data.TFRecordDataset(train_tfrecord_path, 
                                  compression_type='GZIP')
# dataset 길이 확인
dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
print(dataset_length)

train_dataset = train_dataset.map(_parse_image_function, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(map_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# train_dataset = train_dataset.cache()
# dataset shuffle 처리
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

# 전처리(resize)
train_dataset = train_dataset.map(image_resize_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# BatchDataset으로 변환
# <BatchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.float32, tf.int64)>
# BatchDataset으로 변환하기 전에 image의 resize(전처리)가 일어나야 한다. 그렇지 않으면 
# shape이 달라 batch처리가 되지 않는다는 오류 발생.
train_dataset = train_dataset.batch(BATCH_SIZE).repeat()

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
# print(dataset_length)

# prefetch처리
# prefetch는 전처리와 학습과정의 모델 실행을 오버랩.
# 모델이 s스텝 학습을 실행하는 동안 입력 파이프라인은 s+1스텝의 데이터를 읽어서 수행속도를 높임.
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

20000


In [5]:
valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, 
                                  compression_type='GZIP')

# dataset 길이 확인
dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
print(dataset_length)

valid_dataset = valid_dataset.map(_parse_image_function, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# valid_dataset = valid_dataset.cache()
# dataset shuffle 처리
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)

# 전처리(resize)
valid_dataset = valid_dataset.map(image_resize_func, 
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)

# BatchDataset으로 변환
# <BatchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.float32, tf.int64)>
# BatchDataset으로 변환하기 전에 image의 resize(전처리)가 일어나야 한다. 그렇지 않으면 
# shape이 달라 batch처리가 되지 않는다는 오류 발생.
valid_dataset = valid_dataset.batch(BATCH_SIZE).repeat()

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
# print(dataset_length)

# prefetch처리
# prefetch는 전처리와 학습과정의 모델 실행을 오버랩.
# 모델이 s스텝 학습을 실행하는 동안 입력 파이프라인은 s+1스텝의 데이터를 읽어서 수행속도를 높임.
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# # dataset 길이 확인
# dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
# print(dataset_length)

5000


In [ ]:
dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
print(dataset_length)

dataset_length = [i for i,_ in enumerate(valid_dataset)][-1] + 1
print(dataset_length)

In [7]:
# pretrained network
model_base = EfficientNetB3(include_top=False, weights='imagenet', input_shape=(150,150,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten(input_shape=(4*4*512,)))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

model.compile(optimizer=RMSprop(learning_rate=2e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset, steps_per_epoch=100, epochs=20,
                    validation_data=valid_dataset,
                    validation_steps=100)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 5, 5, 1536)        10783535  
_________________________________________________________________
flatten_1 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               9830656   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 20,614,448
Trainable params: 9,830,913
Non-trainable params: 10,783,535
_________________________________________________________________
Epoch 1/20
100/100 [==============================] - 37s 366ms/step - loss: 0.1094 - accuracy: 0.96

In [8]:
model_base.trainable=True

# 상위 layer 동결해제
for layer in model_base.layers:
    if layer.name in ['top_conv', 'block7b_project_conv']:
        layer.trainable=True
    else:
        layer.trainable=False

# 미세조정이므로 learning_rate를 더 작게 설정
model.compile(optimizer=RMSprop(learning_rate=1e-5), loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset, steps_per_epoch=100, epochs=20,
                    validation_data=valid_dataset,
                    validation_steps=100)

Epoch 1/20
100/100 [==============================] - 37s 374ms/step - loss: 0.0139 - accuracy: 0.9952 - val_loss: 0.0493 - val_accuracy: 0.9842
Epoch 2/20
100/100 [==============================] - 36s 364ms/step - loss: 0.0093 - accuracy: 0.9974 - val_loss: 0.0490 - val_accuracy: 0.9840
Epoch 3/20
100/100 [==============================] - 37s 367ms/step - loss: 0.0124 - accuracy: 0.9958 - val_loss: 0.0497 - val_accuracy: 0.9848
Epoch 4/20
100/100 [==============================] - 36s 362ms/step - loss: 0.0071 - accuracy: 0.9980 - val_loss: 0.0513 - val_accuracy: 0.9844
Epoch 5/20
100/100 [==============================] - 39s 389ms/step - loss: 0.0102 - accuracy: 0.9968 - val_loss: 0.0526 - val_accuracy: 0.9842
Epoch 6/20
100/100 [==============================] - 37s 370ms/step - loss: 0.0081 - accuracy: 0.9973 - val_loss: 0.0514 - val_accuracy: 0.9856
Epoch 7/20
100/100 [==============================] - 36s 360ms/step - loss: 0.0096 - accuracy: 0.9975 - val_loss: 0.0552 - val_ac